# Search-8 : CSP Avance - Contraintes globales, OR-Tools et LNS

**Navigation** : [<< Search-7 CSP-Consistency](Search-7-CSP-Consistency.ipynb) | [Index](../README.md) | [Applications >>](../Applications/README.md)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. **Utiliser** les contraintes globales (AllDifferent, Cumulative, Circuit, Table)
2. **Modeliser** des problemes avec Google OR-Tools CP-SAT
3. **Appliquer** des techniques de cassage de symetries pour reduire l'espace de recherche
4. **Comprendre** la recherche a grand voisinage (LNS) pour les problemes de grande taille
5. **Construire** un modele complet combinant contraintes globales et objectif d'optimisation

### Prerequis
- [Search-7 CSP-Consistency](Search-7-CSP-Consistency.ipynb) : AC-3, Forward Checking, MAC
- Bases de Python (classes, dictionnaires, comprehensions)

### Duree estimee : 50 minutes

### Lien avec d'autres series

Ce notebook conclut la sous-serie **Fondations**. Pour voir ces techniques en action sur des problemes du monde reel, explorez la sous-serie [Applications](../Applications/README.md) (planification, ordonnancement, routage). Voir aussi la serie [Sudoku](../../Sudoku/README.md) pour une application complete des CSP.

---

## 1. Introduction (~3 min)

Dans les notebooks precedents (Search-6 et Search-7), nous avons construit nos propres solveurs CSP a la main : backtracking, heuristiques MRV/LCV, propagation AC-3, Forward Checking, MAC. Ces techniques sont fondamentales pour comprendre le fonctionnement interne des solveurs.

Cependant, les **solveurs industriels** modernes vont beaucoup plus loin :

| Aspect | CSP academique (Search-6/7) | Solveur industriel (OR-Tools) |
|--------|---------------------------|-------------------------------|
| Contraintes | Binaires uniquement | Contraintes **globales** specialisees |
| Propagation | AC-3 generique | Propagateurs **dedies** par contrainte |
| Recherche | Backtracking + heuristiques | Recherche hybride (SAT + CP + LNS) |
| Echelle | Dizaines de variables | **Millions** de variables |
| Optimisation | Satisfaction seulement | Satisfaction **et** optimisation |

### Pourquoi les contraintes globales ?

Une contrainte globale comme `AllDifferent(x1, x2, ..., xn)` pourrait etre decomposee en $\binom{n}{2}$ contraintes binaires `xi != xj`. Mais la version globale offre :

1. **Propagation plus forte** : le propagateur AllDifferent detecte des inconsistances invisibles aux contraintes binaires
2. **Meilleure performance** : un seul propagateur optimise plutot que $O(n^2)$ verifications
3. **Modelisation naturelle** : le code est plus lisible et plus proche du probleme reel

### Google OR-Tools CP-SAT

**CP-SAT** (Constraint Programming with SAT) est le solveur de reference de Google OR-Tools. Il combine :
- Programmation par contraintes (CP)
- Solveur SAT (satisfaisabilite booleenne)
- Recherche a grand voisinage (LNS)
- Parallelisme multi-thread

C'est aujourd'hui l'un des solveurs CP les plus performants au monde, regulierement classe premier dans les competitions MiniZinc.

In [ ]:
# Imports pour tout le notebook
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from itertools import permutations, combinations

# python-constraint (CSP academique)
try:
    from constraint import Problem, AllDifferentConstraint
    HAS_CONSTRAINT = True
except ImportError:
    HAS_CONSTRAINT = False
    print("python-constraint non disponible. Installer avec : pip install python-constraint")

# Google OR-Tools (solveur industriel)
try:
    from ortools.sat.python import cp_model
    HAS_ORTOOLS = True
except ImportError:
    HAS_ORTOOLS = False
    print("OR-Tools non disponible. Installer avec : pip install ortools")

# Helpers partages de la serie Search
sys.path.insert(0, '..')
from search_helpers import benchmark_table

print(f"python-constraint : {'OK' if HAS_CONSTRAINT else 'MANQUANT'}")
print(f"OR-Tools          : {'OK' if HAS_ORTOOLS else 'MANQUANT'}")
print("Imports OK")

---

## 2. Contraintes globales (~10 min)

Les **contraintes globales** portent sur un ensemble de variables et possedent des algorithmes de propagation specialises. Voici les quatre contraintes globales les plus utilisees.

| Contrainte | Semantique | Cas d'usage typique |
|------------|-----------|---------------------|
| **AllDifferent** | Toutes les variables ont des valeurs distinctes | Sudoku, emplois du temps |
| **Cumulative** | Somme des ressources utilisees <= capacite a chaque instant | Ordonnancement, planification |
| **Circuit** | Les variables forment un cycle hamiltonien | TSP, routage de vehicules |
| **Table** | Les valeurs doivent appartenir a une liste de tuples autorises | Regles metier, tables de compatibilite |

### 2.1 AllDifferent : la contrainte reine

`AllDifferent(x1, x2, ..., xn)` impose que toutes les variables prennent des valeurs distinctes.

**Pourquoi ne pas simplement poser $x_i \neq x_j$ pour chaque paire ?**

Considerons 3 variables $x_1, x_2, x_3$ avec domaines $\{1, 2\}, \{1, 2\}, \{1, 2\}$. Les contraintes binaires $x_1 \neq x_2$, $x_1 \neq x_3$, $x_2 \neq x_3$ sont toutes arc-consistantes (chaque valeur a un support dans chaque voisin). Pourtant, il n'y a **aucune solution** : 3 variables, seulement 2 valeurs.

Le propagateur AllDifferent (base sur le **theoreme de Hall** et le couplage maximal dans un graphe biparti) detecte immediatement cette inconsistance.

**Exemple concret** : dans un Sudoku, chaque ligne, colonne et bloc 3x3 est une contrainte AllDifferent sur 9 variables.

In [ ]:
# Demonstration AllDifferent : Sudoku simplifie (4x4)
# Comparaison python-constraint vs decomposition binaire

if HAS_CONSTRAINT:
    # Mini-Sudoku 4x4 avec quelques indices
    # +---+---+
    # | 1 . | . . |
    # | . . | 1 . |
    # +---+---+
    # | . 1 | . . |
    # | . . | . 1 |
    # +---+---+
    
    grid_hints = {
        (0, 0): 1,
        (1, 2): 1,
        (2, 1): 1,
        (3, 3): 1,
    }
    
    # Version avec AllDifferent
    p_global = Problem()
    for r in range(4):
        for c in range(4):
            if (r, c) in grid_hints:
                p_global.addVariable((r, c), [grid_hints[(r, c)]])
            else:
                p_global.addVariable((r, c), [1, 2, 3, 4])
    
    # Contraintes AllDifferent par ligne, colonne et bloc 2x2
    for r in range(4):
        p_global.addConstraint(AllDifferentConstraint(), [(r, c) for c in range(4)])
    for c in range(4):
        p_global.addConstraint(AllDifferentConstraint(), [(r, c) for r in range(4)])
    for br in range(2):
        for bc in range(2):
            cells = [(br*2+r, bc*2+c) for r in range(2) for c in range(2)]
            p_global.addConstraint(AllDifferentConstraint(), cells)
    
    start = time.time()
    solutions_global = p_global.getSolutions()
    t_global = (time.time() - start) * 1000
    
    print("Mini-Sudoku 4x4 avec AllDifferent")
    print("=" * 40)
    print(f"Solutions trouvees : {len(solutions_global)}")
    print(f"Temps : {t_global:.2f} ms")
    
    if solutions_global:
        sol = solutions_global[0]
        print("\nPremiere solution :")
        for r in range(4):
            row = [str(sol[(r, c)]) for c in range(4)]
            sep = " | " if True else " "
            print(f"  {row[0]} {row[1]} | {row[2]} {row[3]}")
            if r == 1:
                print(f"  --+--")
else:
    print("python-constraint requis pour cette cellule.")

### Interpretation : AllDifferent sur le Mini-Sudoku

**Sortie obtenue** : le mini-Sudoku 4x4 est resolu avec les contraintes AllDifferent.

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Contraintes AllDifferent | 12 | 4 lignes + 4 colonnes + 4 blocs 2x2 |
| Contraintes binaires equivalentes | 72 | $12 \times \binom{4}{2} = 72$ paires |
| Solutions | Variable | Depend des indices donnes |

**Points cles** :
1. La modelisation AllDifferent est **concise** : 12 contraintes au lieu de 72 binaires
2. Le propagateur AllDifferent est **plus fort** que la decomposition binaire
3. Pour un Sudoku 9x9, la difference est encore plus marquee : 27 AllDifferent vs 810 binaires

### 2.2 Cumulative : ordonnancement sous contraintes de ressources

La contrainte `Cumulative(starts, durations, demands, capacity)` impose que la somme des demandes des taches en cours a chaque instant ne depasse pas la capacite.

$$\forall t : \sum_{i : s_i \leq t < s_i + d_i} r_i \leq C$$

Ou $s_i$ est le debut, $d_i$ la duree, $r_i$ la demande de la tache $i$, et $C$ la capacite.

**Exemple** : 4 taches a planifier sur 2 machines.

In [ ]:
# Demonstration Cumulative : ordonnancement de 4 taches sur 2 machines

# Taches : (nom, duree, ressource_requise)
tasks = [
    ("A", 3, 1),  # Tache A : 3 unites de temps, 1 machine
    ("B", 2, 1),  # Tache B : 2 unites de temps, 1 machine
    ("C", 4, 1),  # Tache C : 4 unites de temps, 1 machine
    ("D", 2, 1),  # Tache D : 2 unites de temps, 1 machine
]
capacity = 2  # 2 machines disponibles en parallele
horizon = 10  # Horizon temporel maximal

if HAS_ORTOOLS:
    model = cp_model.CpModel()
    
    # Variables de debut pour chaque tache
    starts = {}
    ends = {}
    intervals = []
    
    for name, duration, demand in tasks:
        start_var = model.new_int_var(0, horizon - duration, f"start_{name}")
        end_var = model.new_int_var(duration, horizon, f"end_{name}")
        interval_var = model.new_interval_var(start_var, duration, end_var, f"interval_{name}")
        starts[name] = start_var
        ends[name] = end_var
        intervals.append(interval_var)
    
    # Contrainte Cumulative : au plus 2 taches en parallele
    demands = [d for _, _, d in tasks]
    model.add_cumulative(intervals, demands, capacity)
    
    # Objectif : minimiser le makespan (fin de la derniere tache)
    makespan = model.new_int_var(0, horizon, "makespan")
    for name, _, _ in tasks:
        model.add(makespan >= ends[name])
    model.minimize(makespan)
    
    # Resolution
    solver = cp_model.CpSolver()
    status = solver.solve(model)
    
    print("Ordonnancement de 4 taches sur 2 machines")
    print("=" * 50)
    print(f"Statut : {solver.status_name(status)}")
    print(f"Makespan optimal : {solver.value(makespan)} unites de temps")
    print()
    
    schedule = {}
    print(f"{'Tache':<8} {'Debut':>6} {'Fin':>6} {'Duree':>6}")
    print("-" * 30)
    for name, duration, demand in tasks:
        s = solver.value(starts[name])
        e = solver.value(ends[name])
        schedule[name] = (s, e, duration)
        print(f"{name:<8} {s:>6} {e:>6} {duration:>6}")
else:
    print("OR-Tools requis pour cette cellule.")
    schedule = {"A": (0, 3, 3), "B": (0, 2, 2), "C": (2, 6, 4), "D": (3, 5, 2)}

In [ ]:
# Visualisation du planning (diagramme de Gantt)
def draw_gantt(schedule, capacity, title="Diagramme de Gantt"):
    """Affiche un diagramme de Gantt avec profil de charge."""
    colors = ['#2196F3', '#4CAF50', '#FF9800', '#E91E63', '#9C27B0', '#00BCD4']
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 6), 
                             gridspec_kw={'height_ratios': [3, 1]})
    
    # Gantt chart
    ax = axes[0]
    for i, (name, (start, end, duration)) in enumerate(schedule.items()):
        ax.barh(i, duration, left=start, height=0.6,
                color=colors[i % len(colors)], edgecolor='black', linewidth=1.2)
        ax.text(start + duration / 2, i, f"{name}\n[{start}-{end}]",
                ha='center', va='center', fontweight='bold', fontsize=10)
    
    ax.set_yticks(range(len(schedule)))
    ax.set_yticklabels(list(schedule.keys()))
    ax.set_xlabel('Temps')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    ax.axhline(y=-0.5, color='gray', linewidth=0.5)
    
    # Profil de charge
    ax2 = axes[1]
    max_time = max(e for _, e, _ in schedule.values())
    load = np.zeros(max_time + 1)
    for name, (start, end, duration) in schedule.items():
        for t in range(start, end):
            load[t] += 1
    
    ax2.bar(range(max_time + 1), load, color='#90CAF9', edgecolor='#1565C0', linewidth=0.8)
    ax2.axhline(y=capacity, color='red', linestyle='--', linewidth=2, label=f'Capacite = {capacity}')
    ax2.set_xlabel('Temps')
    ax2.set_ylabel('Charge')
    ax2.set_title('Profil de charge des ressources', fontsize=11)
    ax2.legend(fontsize=9)
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

draw_gantt(schedule, capacity, "Ordonnancement optimal - 4 taches, 2 machines")

### Interpretation : contrainte Cumulative

**Sortie obtenue** : le diagramme de Gantt montre l'ordonnancement optimal et le profil de charge.

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Capacite | 2 machines | Au plus 2 taches simultanees |
| Makespan optimal | ~6 unites | Duree totale minimale |
| Profil de charge | Jamais > 2 | La contrainte Cumulative est respectee |

**Points cles** :
1. La contrainte Cumulative gere automatiquement les chevauchements temporels
2. Le solveur optimise le **makespan** (fin de la derniere tache)
3. Sans contrainte Cumulative, il faudrait enumerer les ordres possibles entre taches
4. Dans l'industrie, cette contrainte modelise les lignes de production, les salles d'operation, les pistes d'atterrissage, etc.

### 2.3 Circuit : cycle hamiltonien (TSP)

La contrainte `Circuit(next)` impose que les variables `next[i]` definissent un unique cycle passant par tous les noeuds. Si `next[i] = j`, alors le successeur du noeud $i$ dans le cycle est $j$.

C'est la contrainte cle pour modeliser le **Probleme du Voyageur de Commerce** (TSP) : trouver le circuit le plus court passant par toutes les villes.

In [ ]:
# Demonstration Circuit : petit TSP a 5 villes
cities = ['Paris', 'Lyon', 'Marseille', 'Bordeaux', 'Lille']
n = len(cities)

# Matrice de distances (symetrique)
distances = [
    [0,   465, 775, 585, 225],   # Paris
    [465, 0,   315, 550, 690],   # Lyon
    [775, 315, 0,   645, 1000],  # Marseille
    [585, 550, 645, 0,   810],   # Bordeaux
    [225, 690, 1000, 810, 0],    # Lille
]

if HAS_ORTOOLS:
    model = cp_model.CpModel()
    
    # next[i] = j signifie que la ville suivante apres i est j
    next_vars = [model.new_int_var(0, n - 1, f"next_{i}") for i in range(n)]
    
    # Contrainte Circuit : forme un cycle hamiltonien
    model.add_circuit([(i, next_vars[i]) for i in range(n)])
    
    # Calculer la distance totale
    # Pour chaque arc (i -> next[i]), ajouter la distance correspondante
    arc_costs = []
    for i in range(n):
        cost_var = model.new_int_var(0, max(max(row) for row in distances), f"cost_{i}")
        # cost_var = distances[i][next_vars[i]] via add_element
        model.add_element(next_vars[i], distances[i], cost_var)
        arc_costs.append(cost_var)
    
    total_distance = model.new_int_var(0, sum(max(row) for row in distances), "total")
    model.add(total_distance == sum(arc_costs))
    model.minimize(total_distance)
    
    solver = cp_model.CpSolver()
    status = solver.solve(model)
    
    print("TSP a 5 villes avec contrainte Circuit")
    print("=" * 45)
    print(f"Statut : {solver.status_name(status)}")
    print(f"Distance totale : {solver.value(total_distance)} km")
    print()
    
    # Reconstruire le tour
    tour = [0]
    current = 0
    for _ in range(n - 1):
        nxt = solver.value(next_vars[current])
        tour.append(nxt)
        current = nxt
    
    print("Tour optimal :")
    for i in range(len(tour)):
        j = (i + 1) % len(tour)
        d = distances[tour[i]][tour[j]]
        print(f"  {cities[tour[i]]:>10} -> {cities[tour[j]]:<10} : {d} km")
else:
    print("OR-Tools requis pour cette cellule.")

### Interpretation : contrainte Circuit pour le TSP

**Sortie obtenue** : le solveur trouve le tour optimal passant par les 5 villes.

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Nombre de tours possibles | $(5-1)!/2 = 12$ | Petite instance, mais NP-difficile en general |
| Modelisation | 5 variables + 1 contrainte Circuit | Tres concis |
| Alternative sans Circuit | Sous-tour elimination (MTZ) | Beaucoup plus complexe |

**Points cles** :
1. La contrainte Circuit garantit un **unique cycle** sans sous-tours
2. Sans contrainte globale, il faudrait ajouter des contraintes d'elimination de sous-tours (formulation MTZ ou DFJ)
3. Pour de grandes instances, on combine Circuit avec LNS (section 5)

### 2.4 Table : contraintes extensionnelles

La contrainte `Table(vars, tuples)` impose que les valeurs des variables correspondent a l'un des tuples autorises (ou interdits). C'est utile pour :
- Des regles metier complexes qui ne se decomposent pas en contraintes simples
- Des tables de compatibilite (ex : composants electroniques, emplois du temps)
- Des contraintes apprises a partir de donnees

In [ ]:
# Demonstration Table : compatibilite de composants
# Trois composants (CPU, RAM, Carte mere) avec des combinaisons compatibles

if HAS_ORTOOLS:
    model = cp_model.CpModel()
    
    # Variables : choix de composant (indices)
    # CPU : 0=Intel_i5, 1=Intel_i7, 2=AMD_Ryzen5, 3=AMD_Ryzen7
    # RAM : 0=DDR4_16GB, 1=DDR4_32GB, 2=DDR5_16GB, 3=DDR5_32GB
    # MB  : 0=ASUS_Intel, 1=MSI_Intel, 2=ASUS_AMD, 3=MSI_AMD
    
    cpu = model.new_int_var(0, 3, "cpu")
    ram = model.new_int_var(0, 3, "ram")
    mb = model.new_int_var(0, 3, "mb")
    
    cpu_names = ["Intel i5", "Intel i7", "AMD Ryzen 5", "AMD Ryzen 7"]
    ram_names = ["DDR4 16GB", "DDR4 32GB", "DDR5 16GB", "DDR5 32GB"]
    mb_names = ["ASUS Intel", "MSI Intel", "ASUS AMD", "MSI AMD"]
    
    # Tuples autorises : (cpu, ram, mb)
    # Regle : CPU Intel -> carte mere Intel, CPU AMD -> carte mere AMD
    # Regle : Intel i7 et AMD Ryzen 7 supportent DDR5, les autres DDR4 seulement
    allowed = [
        (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1),  # i5 + DDR4 + Intel MB
        (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1),  # i7 + DDR4 + Intel MB
        (1, 2, 0), (1, 2, 1), (1, 3, 0), (1, 3, 1),  # i7 + DDR5 + Intel MB
        (2, 0, 2), (2, 0, 3), (2, 1, 2), (2, 1, 3),  # Ryzen5 + DDR4 + AMD MB
        (3, 0, 2), (3, 0, 3), (3, 1, 2), (3, 1, 3),  # Ryzen7 + DDR4 + AMD MB
        (3, 2, 2), (3, 2, 3), (3, 3, 2), (3, 3, 3),  # Ryzen7 + DDR5 + AMD MB
    ]
    
    model.add_allowed_assignments([cpu, ram, mb], allowed)
    
    # Enumerer toutes les configurations valides
    solver = cp_model.CpSolver()
    
    class SolutionCounter(cp_model.CpSolverSolutionCallback):
        def __init__(self, variables):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._variables = variables
            self._solutions = []
        
        def on_solution_callback(self):
            self._solutions.append(tuple(self.value(v) for v in self._variables))
    
    callback = SolutionCounter([cpu, ram, mb])
    solver.parameters.enumerate_all_solutions = True
    status = solver.solve(model, callback)
    
    print("Compatibilite de composants PC (contrainte Table)")
    print("=" * 55)
    print(f"Espace total sans contraintes : {4 * 4 * 4} = 64 combinaisons")
    print(f"Combinaisons compatibles      : {len(callback._solutions)}")
    print(f"Reduction                     : {100 - len(callback._solutions)/64*100:.0f}%")
    print()
    print(f"{'CPU':<15} {'RAM':<12} {'Carte mere':<12}")
    print("-" * 40)
    for c, r, m in sorted(callback._solutions):
        print(f"{cpu_names[c]:<15} {ram_names[r]:<12} {mb_names[m]:<12}")
else:
    print("OR-Tools requis pour cette cellule.")

### Interpretation : contrainte Table

**Sortie obtenue** : sur 64 combinaisons possibles, seules 24 sont compatibles.

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Espace total | 64 | $4 \times 4 \times 4$ |
| Combinaisons valides | 24 | 62.5% de reduction |
| Regles encodees | 2 | Compatibilite CPU/MB et support DDR5 |

**Points cles** :
1. La contrainte Table encode des regles metier **arbitraires** sans les decomposer
2. Le propagateur Table est tres efficace : il elimine les valeurs non supportees immediatement
3. En pratique, ces tables peuvent provenir de bases de donnees, de specifications techniques ou d'apprentissage automatique

### Resume des contraintes globales

| Contrainte | Complexite propagation | Force de propagation | Cas d'usage |
|------------|----------------------|---------------------|--------------|
| AllDifferent | $O(n\sqrt{n})$ (Hopcroft-Karp) | Bounds/Domain consistency | Affectation, Sudoku |
| Cumulative | $O(n \log n)$ (sweep) | Timetable + Edge-finding | Ordonnancement |
| Circuit | $O(n)$ (union-find) | Domain consistency | TSP, routage |
| Table | $O(|T| \cdot k)$ | Arc consistency | Regles metier |

---

## 3. OR-Tools CP-SAT Solver (~12 min)

Google OR-Tools CP-SAT est un solveur de **programmation par contraintes** base sur la **satisfaisabilite** (SAT). Il fonctionne en 3 etapes :

1. **Modelisation** : creer un `CpModel`, definir les variables et contraintes
2. **Resolution** : appeler `CpSolver.solve()` (ou avec callback pour les solutions multiples)
3. **Extraction** : lire les valeurs avec `solver.value(var)`

### API principale

| Classe/Methode | Description |
|---------------|-------------|
| `CpModel()` | Creer un modele |
| `model.new_int_var(lb, ub, name)` | Variable entiere |
| `model.new_bool_var(name)` | Variable booleenne |
| `model.add(expr)` | Ajouter une contrainte lineaire |
| `model.add_all_different(vars)` | AllDifferent |
| `model.add_circuit(arcs)` | Circuit |
| `model.add_cumulative(intervals, demands, capacity)` | Cumulative |
| `model.add_allowed_assignments(vars, tuples)` | Table (autorise) |
| `model.add_forbidden_assignments(vars, tuples)` | Table (interdit) |
| `model.minimize(expr)` / `model.maximize(expr)` | Objectif |
| `CpSolver()` | Creer un solveur |
| `solver.solve(model)` | Resoudre |
| `solver.value(var)` | Lire une valeur |

### 3.1 N-Reines avec CP-SAT

Resolvons le probleme des N-Reines avec OR-Tools et comparons la performance avec notre backtracking des notebooks precedents.

In [ ]:
def solve_nqueens_cpsat(n, find_all=False):
    """Resout le probleme des N-Reines avec OR-Tools CP-SAT."""
    if not HAS_ORTOOLS:
        print("OR-Tools requis.")
        return None, 0, 0
    
    model = cp_model.CpModel()
    
    # Variables : queens[i] = ligne de la reine dans la colonne i
    queens = [model.new_int_var(0, n - 1, f"q_{i}") for i in range(n)]
    
    # Contrainte 1 : toutes les reines sur des lignes differentes
    model.add_all_different(queens)
    
    # Contrainte 2 : pas de conflit diagonal
    # q[i] + i != q[j] + j  et  q[i] - i != q[j] - j
    model.add_all_different([queens[i] + i for i in range(n)])
    model.add_all_different([queens[i] - i for i in range(n)])
    
    solver = cp_model.CpSolver()
    
    if find_all:
        class Counter(cp_model.CpSolverSolutionCallback):
            def __init__(self):
                cp_model.CpSolverSolutionCallback.__init__(self)
                self.count = 0
            def on_solution_callback(self):
                self.count += 1
        
        counter = Counter()
        solver.parameters.enumerate_all_solutions = True
        start = time.time()
        status = solver.solve(model, counter)
        elapsed = (time.time() - start) * 1000
        return None, counter.count, elapsed
    else:
        start = time.time()
        status = solver.solve(model)
        elapsed = (time.time() - start) * 1000
        
        if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
            solution = {i: solver.value(queens[i]) for i in range(n)}
            return solution, 1, elapsed
        return None, 0, elapsed


# Resoudre N-Reines pour N = 8
sol_8, count_8, time_8 = solve_nqueens_cpsat(8)

print("N-Reines avec CP-SAT")
print("=" * 40)
print(f"N = 8 : solution trouvee en {time_8:.2f} ms")
print(f"Solution : {sol_8}")

In [ ]:
# Montee en echelle : N = 8 a N = 100
print("Montee en echelle : temps pour trouver UNE solution")
print("=" * 50)
print(f"{'N':>6} {'Temps (ms)':>12} {'Solution':>10}")
print("-" * 35)

scale_results = []
for n in [8, 12, 20, 30, 50, 75, 100]:
    sol, count, t = solve_nqueens_cpsat(n)
    found = "Oui" if sol else "Non"
    scale_results.append((n, t, found))
    print(f"{n:>6} {t:>12.2f} {found:>10}")

print()
print("Comparaison avec notre backtracking (Search-6) :")
print("  - Backtracking + MRV : N=12 peut prendre des secondes")
print("  - CP-SAT : N=100 en quelques millisecondes")

### Interpretation : CP-SAT vs backtracking

**Sortie obtenue** : CP-SAT resout N-Reines jusqu'a N=100 en quelques millisecondes.

| N | CP-SAT (ms) | Backtracking + MRV (estimation) | Facteur |
|---|------------|--------------------------------|----------|
| 8 | < 5 | ~1-5 ms | ~1x |
| 20 | < 10 | ~100 ms | ~10x |
| 50 | < 50 | > 10 s (estime) | > 200x |
| 100 | < 100 | Intraitable | -- |

**Pourquoi CP-SAT est-il si rapide ?**
1. **AllDifferent** : le propagateur specialise est beaucoup plus fort que les contraintes binaires
2. **SAT backend** : la recherche utilise les techniques des solveurs SAT (clause learning, restarts)
3. **Parallelisme** : CP-SAT utilise plusieurs strategies de recherche en parallele
4. **Propagation** : chaque assignation declenche des propagations en chaine tres efficaces

### 3.2 Enumerer toutes les solutions

CP-SAT permet aussi d'enumerer **toutes** les solutions d'un probleme. Comptons les solutions pour differentes tailles de N-Reines.

In [ ]:
# Enumeration de toutes les solutions
print("Nombre de solutions du probleme des N-Reines")
print("=" * 45)
print(f"{'N':>4} {'Solutions':>12} {'Temps (ms)':>12}")
print("-" * 30)

known_solutions = {1: 1, 2: 0, 3: 0, 4: 2, 5: 10, 6: 4, 7: 40, 8: 92,
                   9: 352, 10: 724, 11: 2680, 12: 14200}

for n in range(4, 13):
    _, count, t = solve_nqueens_cpsat(n, find_all=True)
    expected = known_solutions.get(n, '?')
    check = " OK" if count == expected else f" (attendu: {expected})"
    print(f"{n:>4} {count:>12} {t:>12.1f}{check}")

### Interpretation : enumeration des solutions N-Reines

**Sortie obtenue** : CP-SAT enumere toutes les solutions en accord avec les valeurs connues.

| N | Solutions | Croissance |
|---|----------|------------|
| 4 | 2 | -- |
| 8 | 92 | x46 |
| 10 | 724 | x7.9 |
| 12 | 14 200 | x19.6 |

**Points cles** :
1. Le nombre de solutions croit de maniere **super-exponentielle** avec N
2. Le **callback** `on_solution_callback` est appele pour chaque solution sans les stocker toutes en memoire
3. L'enumeration exhaustive reste praticable jusqu'a N ~ 15-16 avec CP-SAT

---

## 4. Cassage de symetries (~8 min)

### Le probleme des symetries

De nombreux CSP possedent des **symetries** : des transformations qui envoient une solution sur une autre solution. Par exemple, pour les N-Reines :

| Symetrie | Transformation | Nb total |
|----------|---------------|----------|
| Rotation 90 | $(i, j) \to (j, n-1-i)$ | 4 |
| Reflexion horizontale | $(i, j) \to (i, n-1-j)$ | 2 |
| Reflexion verticale | $(i, j) \to (n-1-i, j)$ | 2 |
| Groupe dihedral $D_4$ | Toutes les combinaisons | 8 |

Chaque solution a donc jusqu'a **8 variantes symetriques** (certaines solutions sont auto-symetriques). Le solveur perd du temps a explorer ces variantes equivalentes.

### Cassage de symetries

**Principe** : ajouter des contraintes supplementaires qui eliminent les variantes symetriques sans eliminer de solutions fondamentalement differentes.

Pour les N-Reines, une strategie simple :
- Fixer la premiere reine dans la moitie superieure de la premiere colonne : $q_0 < \lceil n/2 \rceil$
- Imposer $q_0 < q_{n-1}$ (elimine la reflexion horizontale)

In [ ]:
def solve_nqueens_with_symmetry_breaking(n, find_all=True):
    """N-Reines avec cassage de symetries."""
    if not HAS_ORTOOLS:
        return 0, 0, 0, 0
    
    # Version SANS cassage de symetries
    model1 = cp_model.CpModel()
    q1 = [model1.new_int_var(0, n - 1, f"q1_{i}") for i in range(n)]
    model1.add_all_different(q1)
    model1.add_all_different([q1[i] + i for i in range(n)])
    model1.add_all_different([q1[i] - i for i in range(n)])
    
    solver1 = cp_model.CpSolver()
    counter1 = type('C', (cp_model.CpSolverSolutionCallback,), {
        '__init__': lambda self: (cp_model.CpSolverSolutionCallback.__init__(self), setattr(self, 'count', 0)),
        'on_solution_callback': lambda self: setattr(self, 'count', self.count + 1)
    })()
    solver1.parameters.enumerate_all_solutions = True
    start = time.time()
    solver1.solve(model1, counter1)
    t1 = (time.time() - start) * 1000
    
    # Version AVEC cassage de symetries
    model2 = cp_model.CpModel()
    q2 = [model2.new_int_var(0, n - 1, f"q2_{i}") for i in range(n)]
    model2.add_all_different(q2)
    model2.add_all_different([q2[i] + i for i in range(n)])
    model2.add_all_different([q2[i] - i for i in range(n)])
    
    # Cassage de symetries
    model2.add(q2[0] < (n + 1) // 2)  # Premiere reine dans la moitie sup
    model2.add(q2[0] < q2[n - 1])     # Elimine reflexion horizontale
    
    solver2 = cp_model.CpSolver()
    counter2 = type('C', (cp_model.CpSolverSolutionCallback,), {
        '__init__': lambda self: (cp_model.CpSolverSolutionCallback.__init__(self), setattr(self, 'count', 0)),
        'on_solution_callback': lambda self: setattr(self, 'count', self.count + 1)
    })()
    solver2.parameters.enumerate_all_solutions = True
    start = time.time()
    solver2.solve(model2, counter2)
    t2 = (time.time() - start) * 1000
    
    return counter1.count, t1, counter2.count, t2


# Comparaison avec et sans cassage de symetries
print("Impact du cassage de symetries sur N-Reines")
print("=" * 65)
print(f"{'N':>4} {'Sans SB':>10} {'Temps':>10} {'Avec SB':>10} {'Temps':>10} {'Ratio sol':>10}")
print("-" * 65)

for n in [6, 8, 10, 11, 12]:
    c1, t1, c2, t2 = solve_nqueens_with_symmetry_breaking(n)
    ratio = f"{c1/c2:.1f}x" if c2 > 0 else "--"
    print(f"{n:>4} {c1:>10} {t1:>9.1f}ms {c2:>10} {t2:>9.1f}ms {ratio:>10}")

### Interpretation : cassage de symetries

**Sortie obtenue** : le cassage de symetries reduit le nombre de solutions et le temps de recherche.

| N | Sans SB | Avec SB | Ratio solutions | Reduction temps |
|---|---------|---------|----------------|------------------|
| 8 | 92 | ~23 | ~4x | Significative |
| 12 | 14 200 | ~3 550 | ~4x | Importante |

**Points cles** :
1. Le ratio est d'environ **4x** car nos deux contraintes eliminent 2 symetries (reflexion + demi-rotation), sur les 8 du groupe dihedral
2. Le gain en temps est **superieur** au ratio de solutions car les branches symetriques eliminees auraient elles-memes genere des echecs couteux
3. Le cassage de symetries est d'autant plus important que le probleme est grand
4. Des techniques plus sophistiquees (SBDS, SBDD) peuvent eliminer **toutes** les symetries automatiquement

### Cas classique : le principe des tiroirs

Le **probleme des pigeons** (pigeonhole principle) illustre l'importance du cassage de symetries : placer $n+1$ pigeons dans $n$ trous, un par trou. C'est impossible, mais prouver l'infaisabilite est exponentiellement couteux sans symetries. Avec cassage de symetries (fixer l'ordre des pigeons), la preuve devient polynomiale.

In [ ]:
# Demonstration : pigeonhole principle avec et sans cassage de symetries
def pigeonhole_test(n_pigeons, n_holes, break_symmetry=False):
    """Tente de placer n_pigeons dans n_holes (1 par trou).
    Retourne (faisable, temps_ms)."""
    if not HAS_ORTOOLS:
        return False, 0
    
    model = cp_model.CpModel()
    # pigeon[i] = trou assigne au pigeon i
    pigeons = [model.new_int_var(0, n_holes - 1, f"p_{i}") for i in range(n_pigeons)]
    model.add_all_different(pigeons)
    
    if break_symmetry:
        # Casser la symetrie des pigeons : imposer un ordre
        for i in range(n_pigeons - 1):
            model.add(pigeons[i] < pigeons[i + 1])
    
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 5.0
    
    start = time.time()
    status = solver.solve(model)
    elapsed = (time.time() - start) * 1000
    
    return status in (cp_model.OPTIMAL, cp_model.FEASIBLE), elapsed


print("Pigeonhole principle : n+1 pigeons dans n trous")
print("=" * 60)
print(f"{'Pigeons':>8} {'Trous':>6} {'Sans SB (ms)':>14} {'Avec SB (ms)':>14} {'Speedup':>8}")
print("-" * 55)

for n in [5, 8, 10, 15, 20]:
    _, t_no_sb = pigeonhole_test(n + 1, n, break_symmetry=False)
    _, t_sb = pigeonhole_test(n + 1, n, break_symmetry=True)
    speedup = f"{t_no_sb / t_sb:.1f}x" if t_sb > 0 else "--"
    print(f"{n+1:>8} {n:>6} {t_no_sb:>13.1f} {t_sb:>13.1f} {speedup:>8}")

print("\nNote : CP-SAT detecte rapidement l'infaisabilite grace a")
print("son propagateur AllDifferent. Le gain est surtout visible")
print("sur des solveurs moins sophistiques.")

---

## 5. Recherche a Grand Voisinage (LNS) (~8 min)

### Quand la recherche complete ne suffit pas

Pour des problemes de tres grande taille (milliers de variables), meme CP-SAT peut ne pas trouver la solution optimale en temps raisonnable. La **Large Neighborhood Search** (LNS) est une meta-heuristique qui :

1. Part d'une solution **initiale** (possiblement mauvaise)
2. **Detruit** une partie de la solution (libere un sous-ensemble de variables)
3. **Re-optimise** le sous-probleme avec un solveur exact (CP-SAT)
4. **Repete** jusqu'a un critere d'arret (temps, iterations, stagnation)

### Schema de LNS

```
Solution initiale S
    |
    v
+---> Destroy : liberer k variables de S
|         |
|         v
|     Repair : re-optimiser le sous-probleme
|         |
|         v
|     Accepter si meilleure (ou avec probabilite)
|         |
+-------- Repeter
```

### CP-SAT et LNS

CP-SAT integre LNS **nativement** via le parallelisme multi-thread. Quand on utilise `num_search_workers > 1`, certains threads executent une recherche LNS tandis que d'autres font une recherche complete. Les solutions trouvees par LNS sont partagees avec les autres threads.

In [ ]:
# Demonstration LNS manuelle sur un probleme d'ordonnancement
# 12 taches a ordonnancer sur 3 machines, minimiser le makespan

import random
random.seed(42)

n_tasks = 12
n_machines = 3
durations = [random.randint(2, 8) for _ in range(n_tasks)]
task_names = [f"T{i}" for i in range(n_tasks)]

print(f"Probleme : {n_tasks} taches sur {n_machines} machines")
print(f"Durees : {durations}")
print(f"Somme des durees : {sum(durations)}")
print(f"Borne inferieure makespan : ceil({sum(durations)}/{n_machines}) = {-(-sum(durations)//n_machines)}")
print()

def solve_scheduling(durations, n_machines, fixed=None, time_limit=1.0):
    """Ordonnancement avec contrainte Cumulative.
    fixed: dict {task_idx: start_time} pour les variables fixees (LNS).
    """
    if not HAS_ORTOOLS:
        return None, 0
    
    n = len(durations)
    horizon = sum(durations)
    model = cp_model.CpModel()
    
    starts = []
    intervals = []
    for i in range(n):
        if fixed and i in fixed:
            s = model.new_int_var(fixed[i], fixed[i], f"s_{i}")
        else:
            s = model.new_int_var(0, horizon - durations[i], f"s_{i}")
        e = model.new_int_var(durations[i], horizon, f"e_{i}")
        interval = model.new_interval_var(s, durations[i], e, f"int_{i}")
        starts.append(s)
        intervals.append(interval)
    
    model.add_cumulative(intervals, [1] * n, n_machines)
    
    makespan = model.new_int_var(0, horizon, "makespan")
    for i in range(n):
        model.add(makespan >= starts[i] + durations[i])
    model.minimize(makespan)
    
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit
    status = solver.solve(model)
    
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        return {i: solver.value(starts[i]) for i in range(n)}, solver.value(makespan)
    return None, float('inf')


# LNS manuelle
def lns_scheduling(durations, n_machines, n_iterations=10, destroy_ratio=0.4, seed=42):
    """LNS pour l'ordonnancement."""
    rng = random.Random(seed)
    n = len(durations)
    k_destroy = max(2, int(n * destroy_ratio))
    
    # Solution initiale (resolution complete)
    current_sol, current_obj = solve_scheduling(durations, n_machines, time_limit=0.5)
    if current_sol is None:
        print("Pas de solution initiale trouvee.")
        return None, float('inf'), []
    
    best_sol = dict(current_sol)
    best_obj = current_obj
    history = [(0, best_obj)]
    
    print(f"Solution initiale : makespan = {best_obj}")
    print(f"Parametres LNS : {n_iterations} iterations, destroy {k_destroy}/{n} variables")
    print()
    
    for it in range(1, n_iterations + 1):
        # Destroy : choisir k variables a liberer
        freed = set(rng.sample(range(n), k_destroy))
        fixed = {i: current_sol[i] for i in range(n) if i not in freed}
        
        # Repair : re-optimiser
        new_sol, new_obj = solve_scheduling(durations, n_machines, fixed=fixed, time_limit=0.3)
        
        if new_sol and new_obj < current_obj:
            current_sol = new_sol
            current_obj = new_obj
            if new_obj < best_obj:
                best_sol = dict(new_sol)
                best_obj = new_obj
                print(f"  Iteration {it:>2} : amelioration -> makespan = {best_obj} (libere: {sorted(freed)})")
        
        history.append((it, best_obj))
    
    return best_sol, best_obj, history


best_sol, best_obj, history = lns_scheduling(durations, n_machines, n_iterations=15)
print(f"\nMeilleur makespan final : {best_obj}")

In [ ]:
# Visualisation de la convergence LNS
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Courbe de convergence
ax1 = axes[0]
iterations = [h[0] for h in history]
objectives = [h[1] for h in history]
ax1.plot(iterations, objectives, 'b-o', linewidth=2, markersize=6)
ax1.axhline(y=best_obj, color='green', linestyle='--', alpha=0.5, label=f'Meilleur = {best_obj}')
lb = -(-sum(durations) // n_machines)
ax1.axhline(y=lb, color='red', linestyle=':', alpha=0.5, label=f'Borne inf = {lb}')
ax1.set_xlabel('Iteration LNS')
ax1.set_ylabel('Makespan')
ax1.set_title('Convergence LNS', fontsize=13, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(alpha=0.3)

# Gantt de la meilleure solution
ax2 = axes[1]
if best_sol:
    colors = plt.cm.tab20(np.linspace(0, 1, n_tasks))
    for i in range(n_tasks):
        ax2.barh(i, durations[i], left=best_sol[i], height=0.7,
                color=colors[i], edgecolor='black', linewidth=0.8)
        ax2.text(best_sol[i] + durations[i] / 2, i, task_names[i],
                ha='center', va='center', fontsize=8, fontweight='bold')
    
    ax2.set_yticks(range(n_tasks))
    ax2.set_yticklabels(task_names, fontsize=8)
    ax2.set_xlabel('Temps')
    ax2.set_title(f'Meilleur ordonnancement (makespan = {best_obj})',
                  fontsize=13, fontweight='bold')
    ax2.axvline(x=best_obj, color='red', linestyle='--', linewidth=2)
    ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### Interpretation : LNS pour l'ordonnancement

**Sortie obtenue** : la courbe de convergence montre l'amelioration progressive du makespan.

| Mesure | Valeur | Signification |
|--------|--------|---------------|
| Solution initiale | Variable | Trouvee par resolution complete |
| Meilleur makespan | Proche de la borne inferieure | LNS ameliore la solution |
| Iterations necessaires | ~5-10 | Convergence rapide |

**Points cles** :
1. LNS converge typiquement vite au debut puis stagne
2. Le choix du **ratio de destruction** (ici 40%) est un parametre crucial :
   - Trop petit : explorations trop locales, convergence lente
   - Trop grand : sous-probleme trop difficile, peu de gain
3. CP-SAT utilise LNS **automatiquement** quand `num_search_workers > 1`
4. En pratique, on utilise LNS via CP-SAT avec un budget de temps :

```python
solver.parameters.max_time_in_seconds = 60.0
solver.parameters.num_search_workers = 8
```

Le solveur alloue automatiquement certains workers a LNS et d'autres a la recherche complete.

---

## 6. Synthese : planification de cours (~6 min)

Combinons tout ce que nous avons appris dans un probleme realiste : la **planification de cours**.

### Enonce

- **5 cours** : Maths, Physique, Info, Anglais, Histoire
- **3 salles** : A, B, C
- **4 creneaux** : Lundi matin, Lundi apres-midi, Mardi matin, Mardi apres-midi
- **Contraintes** :
  - Chaque cours occupe exactement 1 creneau dans 1 salle
  - Pas deux cours dans la meme salle au meme creneau (AllDifferent sur les paires salle-creneau)
  - Maths et Physique ne doivent pas etre au meme creneau (meme prof)
  - Info doit etre en salle C (seule salle avec ordinateurs)
- **Preferences** (objectif) :
  - Maths le matin (bonus +2)
  - Pas de cours le mardi apres-midi si possible (bonus +1 par creneau libre)

In [ ]:
# Modele complet de planification de cours

if HAS_ORTOOLS:
    model = cp_model.CpModel()
    
    # Donnees
    courses = ['Maths', 'Physique', 'Info', 'Anglais', 'Histoire']
    rooms = ['A', 'B', 'C']
    slots = ['Lun_AM', 'Lun_PM', 'Mar_AM', 'Mar_PM']
    n_courses = len(courses)
    n_rooms = len(rooms)
    n_slots = len(slots)
    
    # Variables : room[c] et slot[c] pour chaque cours c
    room_vars = {c: model.new_int_var(0, n_rooms - 1, f"room_{c}") for c in courses}
    slot_vars = {c: model.new_int_var(0, n_slots - 1, f"slot_{c}") for c in courses}
    
    # Contrainte 1 : pas deux cours dans la meme salle au meme creneau
    # On encode la paire (salle, creneau) comme un entier unique : room * n_slots + slot
    room_slot = {}
    for c in courses:
        rs = model.new_int_var(0, n_rooms * n_slots - 1, f"rs_{c}")
        model.add(rs == room_vars[c] * n_slots + slot_vars[c])
        room_slot[c] = rs
    
    model.add_all_different(list(room_slot.values()))
    
    # Contrainte 2 : Maths et Physique pas au meme creneau (meme prof)
    model.add(slot_vars['Maths'] != slot_vars['Physique'])
    
    # Contrainte 3 : Info en salle C (index 2)
    model.add(room_vars['Info'] == 2)
    
    # Objectif : maximiser les preferences
    preference_score = model.new_int_var(0, 100, "preference")
    
    # Maths le matin (creneaux 0 = Lun_AM ou 2 = Mar_AM) -> bonus +2
    maths_morning = model.new_bool_var("maths_morning")
    # maths_morning == 1 si slot_vars['Maths'] est pair (matin)
    maths_slot_mod2 = model.new_int_var(0, 1, "maths_mod2")
    model.add_modulo_equality(maths_slot_mod2, slot_vars['Maths'], 2)
    model.add(maths_slot_mod2 == 0).only_enforce_if(maths_morning)
    model.add(maths_slot_mod2 != 0).only_enforce_if(maths_morning.negated())
    
    # Pas de cours mardi apres-midi (creneau 3) -> bonus +1 par cours non place la
    tue_pm_free = []
    for c in courses:
        not_tue_pm = model.new_bool_var(f"not_tue_pm_{c}")
        model.add(slot_vars[c] != 3).only_enforce_if(not_tue_pm)
        model.add(slot_vars[c] == 3).only_enforce_if(not_tue_pm.negated())
        tue_pm_free.append(not_tue_pm)
    
    model.add(preference_score == 2 * maths_morning + sum(tue_pm_free))
    model.maximize(preference_score)
    
    # Resolution
    solver = cp_model.CpSolver()
    status = solver.solve(model)
    
    print("Planification de cours - Modele complet")
    print("=" * 55)
    print(f"Statut    : {solver.status_name(status)}")
    print(f"Preference: {solver.value(preference_score)} (max possible: {2 + n_courses})")
    print()
    
    planning = {}
    print(f"{'Cours':<12} {'Salle':<8} {'Creneau':<12}")
    print("-" * 35)
    for c in courses:
        r = solver.value(room_vars[c])
        s = solver.value(slot_vars[c])
        planning[c] = (rooms[r], slots[s])
        print(f"{c:<12} {rooms[r]:<8} {slots[s]:<12}")
else:
    print("OR-Tools requis pour cette cellule.")
    courses = ['Maths', 'Physique', 'Info', 'Anglais', 'Histoire']
    rooms = ['A', 'B', 'C']
    slots = ['Lun_AM', 'Lun_PM', 'Mar_AM', 'Mar_PM']
    planning = {}

In [ ]:
# Visualisation : grille emploi du temps
def draw_timetable(planning, rooms, slots, title="Emploi du temps"):
    """Affiche l'emploi du temps sous forme de grille."""
    fig, ax = plt.subplots(figsize=(10, 5))
    
    n_rooms = len(rooms)
    n_slots = len(slots)
    
    colors = {
        'Maths': '#2196F3',
        'Physique': '#4CAF50',
        'Info': '#FF9800',
        'Anglais': '#E91E63',
        'Histoire': '#9C27B0',
    }
    
    # Grille vide
    for r in range(n_rooms):
        for s in range(n_slots):
            rect = plt.Rectangle((s, n_rooms - 1 - r), 1, 1,
                                  facecolor='#FAFAFA', edgecolor='#B0BEC5', linewidth=1.5)
            ax.add_patch(rect)
    
    # Placer les cours
    for course, (room, slot) in planning.items():
        r_idx = rooms.index(room)
        s_idx = slots.index(slot)
        rect = plt.Rectangle((s_idx + 0.05, n_rooms - 1 - r_idx + 0.05), 0.9, 0.9,
                              facecolor=colors.get(course, '#BDBDBD'),
                              edgecolor='black', linewidth=2, alpha=0.85)
        ax.add_patch(rect)
        ax.text(s_idx + 0.5, n_rooms - 1 - r_idx + 0.5, course,
                ha='center', va='center', fontsize=11, fontweight='bold', color='white')
    
    # Labels
    ax.set_xticks([s + 0.5 for s in range(n_slots)])
    ax.set_xticklabels([s.replace('_', '\n') for s in slots], fontsize=10)
    ax.set_yticks([r + 0.5 for r in range(n_rooms)])
    ax.set_yticklabels(list(reversed(rooms)), fontsize=11)
    ax.set_ylabel('Salle', fontsize=12)
    ax.set_xlabel('Creneau', fontsize=12)
    ax.set_xlim(0, n_slots)
    ax.set_ylim(0, n_rooms)
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()

if planning:
    draw_timetable(planning, rooms, slots,
                   "Emploi du temps optimal")
else:
    print("Pas de solution a visualiser.")

### Interpretation : planification de cours

**Sortie obtenue** : un emploi du temps complet respectant toutes les contraintes et maximisant les preferences.

| Contrainte | Respectee |
|------------|--------|
| Pas de conflit salle-creneau | Oui (AllDifferent) |
| Maths et Physique separes | Oui (creneaux differents) |
| Info en salle C | Oui |
| Maths le matin | Selon le score de preference |

**Points cles** :
1. Le modele combine **contraintes dures** (obligatoires) et **contraintes souples** (preferences via l'objectif)
2. La technique d'encodage `room * n_slots + slot` transforme une contrainte 2D en contrainte 1D pour AllDifferent
3. Les `BoolVar` et `only_enforce_if` permettent de modeliser des preferences conditionnelles
4. Ce type de modele se generalise aux emplois du temps reels avec des centaines de cours et de contraintes

> **Pour aller plus loin** : voir la sous-serie [Applications](../Applications/README.md) pour des problemes de planification a grande echelle (Nurse Scheduling, Job-Shop, Timetabling).

---

## 7. Resume et exercices

### Tableau recapitulatif : contraintes globales

| Contrainte | Semantique | Cas d'usage | Support OR-Tools |
|------------|-----------|-------------|------------------|
| **AllDifferent** | Toutes distinctes | Sudoku, emploi du temps | `add_all_different` |
| **Cumulative** | Charge <= capacite | Ordonnancement | `add_cumulative` |
| **Circuit** | Cycle hamiltonien | TSP, routage | `add_circuit` |
| **Table** | Tuples autorises/interdits | Regles metier | `add_allowed_assignments` |
| **Element** | Indexation de tableau | Couts variables | `add_element` |
| **NoOverlap** | Pas de chevauchement | Planning | `add_no_overlap` |

### Tableau recapitulatif : techniques avancees

| Technique | Principe | Quand l'utiliser |
|-----------|---------|------------------|
| **Cassage de symetries** | Eliminer les solutions equivalentes | Toujours, si des symetries existent |
| **LNS** | Detruire/reparer partiellement | Problemes de grande taille, optimisation |
| **Parallelisme** | Strategies multiples en parallele | Toujours (defaut CP-SAT) |
| **Bornes** | Borne inferieure pour prouver l'optimalite | Verification de la qualite |

### Bilan de la sous-serie CSP (Search-6, 7, 8)

| Notebook | Contenu | Technique cle |
|----------|---------|---------------|
| Search-6 | Formalisme CSP, backtracking, heuristiques | MRV, LCV, Degree |
| Search-7 | Propagation de contraintes | AC-3, Forward Checking, MAC |
| **Search-8** | Contraintes globales, solveurs industriels, LNS | OR-Tools CP-SAT, AllDifferent, Cumulative |

### Et maintenant ?

Ce notebook conclut la sous-serie **Fondations** de la serie Search. Vous maitrisez maintenant les concepts fondamentaux de la recherche et de la programmation par contraintes.

Pour mettre ces connaissances en pratique, explorez :
- La sous-serie **[Applications](../Applications/README.md)** : N-Queens a grande echelle, coloration de graphes, Nurse Scheduling, Job-Shop, Minesweeper, Wordle, MiniZinc
- La serie **[Sudoku](../../Sudoku/README.md)** : application complete des CSP avec 6+ solveurs differents

### References

- Rossi, F., van Beek, P., Walsh, T. *Handbook of Constraint Programming*, Elsevier, 2006
- Google OR-Tools : [documentation officielle](https://developers.google.com/optimization)
- Perron, L. & Furnon, V. *OR-Tools CP-SAT Solver*, Google, 2023
- Russell, S. & Norvig, P. *Artificial Intelligence: A Modern Approach*, Chapitre 6

### Exercice 1 : SEND + MORE = MONEY avec CP-SAT

**Enonce** : modelisez le cryptarithme SEND + MORE = MONEY avec OR-Tools CP-SAT.

Rappel : chaque lettre represente un chiffre different (0-9), S et M ne sont pas 0.

$$\text{SEND} + \text{MORE} = \text{MONEY}$$

Comparez avec la version backtracking de Search-6 en termes de lignes de code et de temps d'execution.

In [ ]:
# Exercice 1 : SEND + MORE = MONEY avec CP-SAT

# A COMPLETER
# model = cp_model.CpModel()
# S = model.new_int_var(1, 9, 'S')  # S != 0
# E = model.new_int_var(0, 9, 'E')
# N = model.new_int_var(0, 9, 'N')
# D = model.new_int_var(0, 9, 'D')
# M = model.new_int_var(1, 9, 'M')  # M != 0
# O = model.new_int_var(0, 9, 'O')
# R = model.new_int_var(0, 9, 'R')
# Y = model.new_int_var(0, 9, 'Y')
#
# model.add_all_different([S, E, N, D, M, O, R, Y])
# SEND  = S * 1000 + E * 100 + N * 10 + D
# MORE  = M * 1000 + O * 100 + R * 10 + E
# MONEY = M * 10000 + O * 1000 + N * 100 + E * 10 + Y
# model.add(SEND + MORE == MONEY)
#
# solver = cp_model.CpSolver()
# status = solver.solve(model)
# print(f"{solver.value(S)}{solver.value(E)}{solver.value(N)}{solver.value(D)}")
# print(f"+ {solver.value(M)}{solver.value(O)}{solver.value(R)}{solver.value(E)}")
# print(f"= {solver.value(M)}{solver.value(O)}{solver.value(N)}{solver.value(E)}{solver.value(Y)}")

<details>
<summary><b>Solution exercice 1</b></summary>

```python
model = cp_model.CpModel()
S = model.new_int_var(1, 9, 'S')
E = model.new_int_var(0, 9, 'E')
N = model.new_int_var(0, 9, 'N')
D = model.new_int_var(0, 9, 'D')
M = model.new_int_var(1, 9, 'M')
O = model.new_int_var(0, 9, 'O')
R = model.new_int_var(0, 9, 'R')
Y = model.new_int_var(0, 9, 'Y')

model.add_all_different([S, E, N, D, M, O, R, Y])

SEND  = S * 1000 + E * 100 + N * 10 + D
MORE  = M * 1000 + O * 100 + R * 10 + E
MONEY = M * 10000 + O * 1000 + N * 100 + E * 10 + Y
model.add(SEND + MORE == MONEY)

solver = cp_model.CpSolver()
start = time.time()
status = solver.solve(model)
elapsed = (time.time() - start) * 1000

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    vals = {v.name: solver.value(v) for v in [S, E, N, D, M, O, R, Y]}
    print(f"  {vals['S']}{vals['E']}{vals['N']}{vals['D']}")
    print(f"+ {vals['M']}{vals['O']}{vals['R']}{vals['E']}")
    print(f"------")
    print(f"{vals['M']}{vals['O']}{vals['N']}{vals['E']}{vals['Y']}")
    print(f"\nTemps : {elapsed:.2f} ms")
```

La solution est : 9567 + 1085 = 10652.

Comparaison avec Search-6 : le modele CP-SAT est plus concis (pas besoin d'implementer le backtracking) et plus rapide.

</details>

### Exercice 2 : Cassage de symetries pour N-Reines

**Enonce** : ajoutez des contraintes de cassage de symetries supplementaires au modele N-Reines CP-SAT pour eliminer plus de symetries du groupe dihedral $D_4$.

Indices :
- Le groupe $D_4$ a 8 elements (4 rotations + 4 reflexions)
- Pour eliminer les reflexions verticales : $q_0 < q_{n-1}$
- Pour eliminer les rotations : $q_0 \leq q_{n-1-q_0}$ (lex-leader)

Mesurez le nombre de solutions "fondamentales" (sans symetries) pour N=8 et comparez avec les 92 solutions totales.

In [ ]:
# Exercice 2 : Cassage de symetries avance pour N-Reines

# A COMPLETER
# Conseil : commencez avec les contraintes de la section 4,
# puis ajoutez des contraintes supplementaires.
# Resultat attendu pour N=8 : 12 solutions fondamentales (sur 92 totales).
#
# model = cp_model.CpModel()
# n = 8
# q = [model.new_int_var(0, n - 1, f"q_{i}") for i in range(n)]
# model.add_all_different(q)
# model.add_all_different([q[i] + i for i in range(n)])
# model.add_all_different([q[i] - i for i in range(n)])
#
# # Ajoutez vos contraintes de cassage de symetries ici
# ...
#
# # Comptez les solutions
# ...

<details>
<summary><b>Solution exercice 2</b></summary>

```python
model = cp_model.CpModel()
n = 8
q = [model.new_int_var(0, n - 1, f"q_{i}") for i in range(n)]
model.add_all_different(q)
model.add_all_different([q[i] + i for i in range(n)])
model.add_all_different([q[i] - i for i in range(n)])

# Cassage de symetries : fixer q[0] dans le premier quart
model.add(q[0] < (n + 1) // 2)
# Eliminer reflexion horizontale
model.add(q[0] < q[n - 1])
# Eliminer reflexion diagonale
model.add(q[0] <= q[n - 1 - 0])  # q[0] <= q[n-1]

solver = cp_model.CpSolver()

class Counter(cp_model.CpSolverSolutionCallback):
    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.count = 0
    def on_solution_callback(self):
        self.count += 1

counter = Counter()
solver.parameters.enumerate_all_solutions = True
solver.solve(model, counter)

print(f"Solutions fondamentales (N=8) : {counter.count}")
print(f"Solutions totales : 92")
print(f"Ratio : 92 / {counter.count} = {92 / counter.count:.1f}")
```

On obtient environ 12-23 solutions fondamentales selon le niveau de cassage de symetries. Les 12 solutions veritablement uniques (module le groupe dihedral complet $D_4$) necessitent un cassage complet.

</details>

### Exercice 3 : Mini-VRP avec contrainte Circuit

**Enonce** : modelisez un probleme simplifie de tournees de vehicules (VRP).

- 1 depot (noeud 0) et 4 clients (noeuds 1-4)
- 1 vehicule qui doit visiter tous les clients et revenir au depot
- Distances donnees ci-dessous
- Objectif : minimiser la distance totale

Utilisez la contrainte `Circuit` de CP-SAT.

In [ ]:
# Exercice 3 : Mini-VRP avec Circuit

# Matrice de distances (depot = noeud 0, clients = noeuds 1-4)
vrp_distances = [
    [0,  10, 15, 20, 25],  # Depot
    [10,  0, 35, 25, 30],  # Client 1
    [15, 35,  0, 30, 20],  # Client 2
    [20, 25, 30,  0, 15],  # Client 3
    [25, 30, 20, 15,  0],  # Client 4
]

# A COMPLETER
# model = cp_model.CpModel()
# n = 5
# next_vars = [model.new_int_var(0, n - 1, f"next_{i}") for i in range(n)]
# model.add_circuit([(i, next_vars[i]) for i in range(n)])
#
# # Calculer la distance totale
# arc_costs = []
# for i in range(n):
#     cost = model.new_int_var(0, max(max(r) for r in vrp_distances), f"cost_{i}")
#     model.add_element(next_vars[i], vrp_distances[i], cost)
#     arc_costs.append(cost)
#
# total = sum(arc_costs)
# model.minimize(total)
#
# solver = cp_model.CpSolver()
# status = solver.solve(model)
# print(f"Distance totale : {solver.objective_value}")

<details>
<summary><b>Solution exercice 3</b></summary>

```python
model = cp_model.CpModel()
n = 5  # depot + 4 clients

next_vars = [model.new_int_var(0, n - 1, f"next_{i}") for i in range(n)]
model.add_circuit([(i, next_vars[i]) for i in range(n)])

arc_costs = []
for i in range(n):
    cost = model.new_int_var(0, max(max(r) for r in vrp_distances), f"cost_{i}")
    model.add_element(next_vars[i], vrp_distances[i], cost)
    arc_costs.append(cost)

total_dist = model.new_int_var(0, sum(max(r) for r in vrp_distances), "total")
model.add(total_dist == sum(arc_costs))
model.minimize(total_dist)

solver = cp_model.CpSolver()
status = solver.solve(model)

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    nodes = ['Depot', 'Client 1', 'Client 2', 'Client 3', 'Client 4']
    print(f"Distance totale : {solver.value(total_dist)}")
    current = 0
    for _ in range(n):
        nxt = solver.value(next_vars[current])
        d = vrp_distances[current][nxt]
        print(f"  {nodes[current]} -> {nodes[nxt]} ({d} km)")
        current = nxt
```

Le tour optimal a une distance d'environ 80 km.

</details>

---

**Navigation** : [<< Search-7 CSP-Consistency](Search-7-CSP-Consistency.ipynb) | [Index](../README.md) | [Applications >>](../Applications/README.md)